In [14]:
import polars as pl 
import json 
import numpy as np

from sklearn.model_selection import cross_validate, TimeSeriesSplit
from xgboost import XGBClassifier

In [15]:
# Changer l'adresse des fichiers si besoin 
transactions = pl.read_parquet("transactions_post_feature_engineering.parquet")
with open("features_used.json", "r") as f:
    feature_names = json.load(f)


selected_region = "nom_region_Occitanie"
region_transactions = transactions.filter(pl.col(selected_region) == 1)

In [16]:

X = region_transactions.drop([REGRESSION_TARGET, CLASSIFICATION_TARGET])
y_classification = region_transactions[CLASSIFICATION_TARGET]


In [17]:
def perform_cross_validation(
    X: pl.DataFrame,
    y: pl.Series,
    model,
    cross_val_type,
    scoring_metrics: tuple,
    return_estimator=False,
    groups=None,
):
    scores = cross_validate(
        model,
        X.to_numpy(),
        y.to_numpy(),
        cv=cross_val_type,
        return_train_score=True,
        return_estimator=return_estimator,
        scoring=scoring_metrics,
        groups=groups,
    )

    for metric in scoring_metrics:
        print(
            "Average Train {metric} : {metric_value}".format(
                metric=metric,
                metric_value=np.mean(scores["train_" + metric]),
            )
        )
        print(
            "Train {metric} Standard Deviation : {metric_value}".format(
                metric=metric, metric_value=np.std(scores["train_" + metric])
            )
        )

        print(
            "Average Test {metric} : {metric_value}".format(
                metric=metric, metric_value=np.mean(scores["test_" + metric])
            )
        )
        print(
            "Test {metric} Standard Deviation : {metric_value}".format(
                metric=metric, metric_value=np.std(scores["test_" + metric])
            )
        )

    return scores